In [2]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot
import tensorflow as tf
import datetime
import logging
import os
import io
import numpy as np

Using TensorFlow backend.


In [3]:

            

# #Hyperparameter Tuning with the HParams Dashboard
# from tensorboard.plugins.hparams import api as hp

# HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

# METRIC_ACCURACY = 'accuracy'

# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# 	hp.hparams_config(
# 		hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
# 		metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
# 	)

    
# def train_test_model(hparams):
# 	model = tf.keras.models.Sequential([
# 		tf.keras.layers.LSTM(100, input_shape=(n_timesteps,n_features)),
# 		tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
# 		tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
# 		tf.keras.layers.Dense(10, activation=tf.nn.softmax),
# 	])
# 	model.compile(
# 		optimizer=hparams[HP_OPTIMIZER],
# 		loss='sparse_categorical_crossentropy',
# 		metrics=['accuracy'],
# 	)

# def run(run_dir, hparams):
# 	with tf.summary.create_file_writer(run_dir).as_default():
# 		hp.hparams(hparams)  # record the values used in this trial
# 		accuracy = train_test_model(hparams)
# 		tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy


def create_model(n_timesteps,n_features,n_outputs):
	return tf.keras.models.Sequential([
		tf.keras.layers.LSTM(100, input_shape=(n_timesteps,n_features)),
		tf.keras.layers.Dropout(0.5),
		tf.keras.layers.Dense(100, activation='relu'),
		tf.keras.layers.Dense(n_outputs, activation='softmax')
	])


# # fit and evaluate a model
# def evaluate_model(trainX, trainy, testX, testy):
# 	# define model
# 	verbose, epochs, batch_size = 0, 25, 64
# 	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
# 	# reshape data into time steps of sub-sequences
# 	n_steps, n_length = 4, 32
# 	trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
# 	testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
# 	# define model
# #	model = Sequential()
# 	#model = tf.keras.Sequential(n_length, n_features)
# 	model = create_model(n_length, n_features, n_outputs)

# #	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
# #	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
# #	model.add(TimeDistributed(Dropout(0.5)))
# #	model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
# #	model.add(TimeDistributed(Flatten()))
# #	model.add(LSTM(100))
# #	model.add(Dropout(0.5))
# #	model.add(Dense(100, activation='relu'))
# #	model.add(Dense(n_outputs, activation='softmax'))
# 	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	print("FIT ==================================================1 ");
# 	log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# 	tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# 	# fit network
# # 	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
# # 	print("FIT ==================================================2 ");
# # 	# evaluate model
# # 	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    
#     ## epoches = how many times run - changed to for statement.
#     ## verbose = log depth; 0=small amount logging. 1= show Epoch 1/1 7352/7352
#     ## batch_size == 
# 	for i in range(epochs):
# 		print("FIT ==================================================2 ");
# 		#model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
# #model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=1)
# #		model.fit(trainX, trainy, batch_size=batch_size, verbose=1)
# 		model.fit(trainX, trainy, batch_size=batch_size, verbose=1, callbacks=[tensorboard_callback])
# # 		##model.reset_states()
# 		print("FIT ==================================================3 ");
# 	print("Evalutate ================================================== ");
# 	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1, shuffle=False)
# 	print("Evalutate ================================================== ");
# 	return accuracy


# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
#	verbose, epochs, batch_size = 0, 15, 64
	verbose, epochs, batch_size = 0, 5, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
# 	model = Sequential()
# 	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
# 	model.add(Dropout(0.5))
# 	model.add(Dense(100, activation='relu'))
# 	model.add(Dense(n_outputs, activation='softmax'))



#for graph1
	model = create_model(n_timesteps,n_features,n_outputs)
# 	model= tf.keras.models.Sequential([
# 		tf.keras.layers.LSTM(100, input_shape=(n_timesteps,n_features)),
# 		tf.keras.layers.Dropout(0.5),
# 		tf.keras.layers.Dense(100, activation='relu'),
# 		tf.keras.layers.Dense(n_outputs, activation='softmax')
# 	])


# get TF logger
	log = logging.getLogger('tensorflow')
	log.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
#	formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

#	log_dir = "logs/test/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#	log_file = '{}/dashboard.log'.format(log_loc)
#	Dir = os.path.join(os.path.dirname(os.path.abspath(logname.dname)), 'logs')
	formatter = logging.Formatter('%(asctime)s %(levelname)s:%(name)s %(message)s')
	logFile = 'Log %s.log' % (datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S'))
	Dirfilelog = os.path.join('logs/', logFile)
    
# create file handler which logs even debug messages
	fh = logging.FileHandler(Dirfilelog)
	fh.setLevel(logging.DEBUG)
	fh.setFormatter(formatter)
	log.addHandler(fh)

	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #for Graph1
	#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    #for Graph2
#	tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)


	def get_model_summary(model):
		stream = io.StringIO()
		model.summary(print_fn=lambda x: stream.write(x + '\n'))
		summary_string = stream.getvalue()
		stream.close()
		return summary_string

	model_summary_string = get_model_summary(model)
	log.info(model_summary_string)

    #for customGraph
	class MyCustomCallback(tf.keras.callbacks.Callback):
#		def set_model(self, batch, logs=None):
# 			model_summary_string = get_model_summary(model)
# 			log.info(model_summary_string)
		def on_train_batch_end(self, batch, logs=None):
			log.info('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))
			log.info('For batch {}, accuracy is {:7.2f}.'.format(batch, logs['accuracy']))
		def on_test_batch_end(self, batch, logs=None):
			log.info('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))
			log.info('For batch {}, accuracy is {:7.2f}.'.format(batch, logs['accuracy']))
		def on_epoch_end(self, epoch, logs=None):
			#np.testing.assert_allclose(np.squeeze(out), np.squeeze(out2), atol=1e-05)
			for i in range(len(model.layers)):
				new_weights = model.layers[i].get_weights()
				log.info('weights of {} layers for epoch{} is {}'.format(i+1, epoch, new_weights))     
				#for j in range(len(new_weights)):
					#if old_weights[i]:
						#np.testing.assert_allclose(old_weights[i][j], new_weights[j], atol=1e-05)     
			log.info('The average loss for epoch {} is {:7.2f} and accuracy is {:7.2f}.'.format(epoch, logs['loss'], logs['accuracy']))

           

	# fit network
# 	for i in range(epochs):
# 		model.fit(trainX, trainy, batch_size=batch_size, verbose=1)
        
    #For graph1
	#model.fit(x=trainX, y=trainy, epochs=epochs, verbose=1, validation_data=(testX, testy), callbacks=[tensorboard_callback])
    
    
    #For graph2
#	model.fit(x=trainX, y=trainy, epochs=epochs,batch_size=64, verbose=1, callbacks=[LossAndErrorPrintingCallback()])

    #For CustomGraph
    
#	test.test1(model);

	model.fit(x=trainX, y=trainy, epochs=epochs,batch_size=64, verbose=1, callbacks=[MyCustomCallback()])
#	model.fit(x=trainX, y=trainy, epochs=epochs,batch_size=64, verbose=1, callbacks=[test.test1(test, epochs, model)])
    
    #for Graph3
# 	model.fit(trainX, trainy, epochs=1,
# 		callbacks=[
# 		tf.keras.callbacks.TensorBoard(logdir),  # log metrics
# 		hp.KerasCallback(logdir, hparams),  # log hparams
# 	],) # Run with 1 epoch to speed things up for demo purposes
    
# 		model.fit(x=trainX, y=trainy, validation_data=(testX, testy), callbacks=[tensorboard_callback])
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
	return accuracy

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
#def run_experiment(repeats=10):
def run_experiment(repeats=1):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

# 	session_num = 0
# 	for num_units in HP_NUM_UNITS.domain.values:
# 		for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
# 			for optimizer in HP_OPTIMIZER.domain.values:
# 				hparams = {
# 					HP_NUM_UNITS: num_units,
# 					HP_DROPOUT: dropout_rate,
# 					HP_OPTIMIZER: optimizer,
# 				}
# 				run_name = "run-%d" % session_num
# 				print('--- Starting trial: %s' % run_name)
# 				print({h.name: hparams[h] for h in hparams})
# 				run('logs/hparam_tuning/' + run_name, hparams)
# 				session_num += 1
    
# def get_model_summary(model):
# 	stream = io.StringIO()
# 	model.summary(print_fn=lambda x: stream.write(x + '\n'))
# 	summary_string = stream.getvalue()
# 	stream.close()
# 	return summary_string


# run the experiment
run_experiment()


        

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
INFO:tensorflow:Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               44000     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 606       
Total params: 54,706
Trainable params: 54,706
Non-trainable params: 0
_________________________________________________________________

Train on 7352 samples
Epoch 1/5
INFO:tensorflow:For batch 0, loss is    1.76.
INFO:tensorflow:For batch 0, accuracy is    0.28.
  64/

2496/7352 [=========>....................] - ETA: 19s - loss: 1.3975 - accuracy: 0.4187INFO:tensorflow:For batch 39, loss is    1.08.
INFO:tensorflow:For batch 39, accuracy is    0.42.
2560/7352 [=========>....................] - ETA: 18s - loss: 1.3896 - accuracy: 0.4223INFO:tensorflow:For batch 40, loss is    1.26.
INFO:tensorflow:For batch 40, accuracy is    0.42.
2624/7352 [=========>....................] - ETA: 18s - loss: 1.3864 - accuracy: 0.4238INFO:tensorflow:For batch 41, loss is    1.25.
INFO:tensorflow:For batch 41, accuracy is    0.42.
2688/7352 [=========>....................] - ETA: 18s - loss: 1.3832 - accuracy: 0.4241INFO:tensorflow:For batch 42, loss is    0.95.
INFO:tensorflow:For batch 42, accuracy is    0.43.
2752/7352 [==========>...................] - ETA: 17s - loss: 1.3731 - accuracy: 0.4277INFO:tensorflow:For batch 43, loss is    1.22.
INFO:tensorflow:For batch 43, accuracy is    0.43.
2816/7352 [==========>...................] - ETA: 17s - loss: 1.3697 - accu

5312/7352 [====================>.........] - ETA: 7s - loss: 1.2265 - accuracy: 0.4825INFO:tensorflow:For batch 83, loss is    1.02.
INFO:tensorflow:For batch 83, accuracy is    0.48.
5376/7352 [====================>.........] - ETA: 6s - loss: 1.2241 - accuracy: 0.4846INFO:tensorflow:For batch 84, loss is    1.11.
INFO:tensorflow:For batch 84, accuracy is    0.49.
5440/7352 [=====================>........] - ETA: 6s - loss: 1.2227 - accuracy: 0.4853INFO:tensorflow:For batch 85, loss is    0.95.
INFO:tensorflow:For batch 85, accuracy is    0.49.
5504/7352 [=====================>........] - ETA: 6s - loss: 1.2195 - accuracy: 0.4866INFO:tensorflow:For batch 86, loss is    0.88.
INFO:tensorflow:For batch 86, accuracy is    0.49.
5568/7352 [=====================>........] - ETA: 6s - loss: 1.2156 - accuracy: 0.4880INFO:tensorflow:For batch 87, loss is    1.03.
INFO:tensorflow:For batch 87, accuracy is    0.49.
5632/7352 [=====================>........] - ETA: 5s - loss: 1.2135 - accuracy: 

INFO:tensorflow:weights of 2 layers for epoch0 is []
INFO:tensorflow:weights of 3 layers for epoch0 is [array([[-0.02672616,  0.14621186,  0.05128063, ..., -0.10931678,
         0.04192034,  0.03566625],
       [-0.04017503, -0.16816749, -0.08515023, ...,  0.082712  ,
        -0.17125043,  0.09562171],
       [ 0.05237131,  0.17473064, -0.03619789, ...,  0.12029404,
         0.15826368, -0.05491549],
       ...,
       [ 0.13180546,  0.06070846, -0.22198531, ..., -0.11049569,
         0.05071298,  0.12191661],
       [ 0.0365625 ,  0.05445123, -0.15149467, ..., -0.13128184,
         0.11104594, -0.00207009],
       [ 0.13779852,  0.18599337, -0.06132601, ...,  0.04542718,
        -0.10317281,  0.07537553]], dtype=float32), array([ 6.08346472e-03,  1.28350351e-02,  1.81909148e-02,  4.34350967e-02,
        1.26857515e-02,  2.07579918e-02, -2.65386095e-03, -2.96354922e-03,
       -1.32720433e-02,  2.31985319e-02,  4.20463905e-02,  3.05860769e-02,
        4.61909594e-03,  2.36460473e-02,  

INFO:tensorflow:The average loss for epoch 0 is    1.13 and accuracy is    0.52.
7352/7352 [==============================] - 24s 3ms/sample - loss: 1.1324 - accuracy: 0.5227
Epoch 2/5
INFO:tensorflow:For batch 0, loss is    0.80.
INFO:tensorflow:For batch 0, accuracy is    0.59.
  64/7352 [..............................] - ETA: 17s - loss: 0.7983 - accuracy: 0.5938INFO:tensorflow:For batch 1, loss is    0.69.
INFO:tensorflow:For batch 1, accuracy is    0.62.
 128/7352 [..............................] - ETA: 16s - loss: 0.7460 - accuracy: 0.6250INFO:tensorflow:For batch 2, loss is    0.90.
INFO:tensorflow:For batch 2, accuracy is    0.60.
 192/7352 [..............................] - ETA: 16s - loss: 0.7962 - accuracy: 0.6042INFO:tensorflow:For batch 3, loss is    0.77.
INFO:tensorflow:For batch 3, accuracy is    0.62.
 256/7352 [>.............................] - ETA: 16s - loss: 0.7906 - accuracy: 0.6211INFO:tensorflow:For batch 4, loss is    0.72.
INFO:tensorflow:For batch 4, accuracy

...

INFO:tensorflow:The average loss for epoch 3 is    0.48 and accuracy is    0.82.
7352/7352 [==============================] - 22s 3ms/sample - loss: 0.4817 - accuracy: 0.8161
Epoch 5/5
INFO:tensorflow:For batch 0, loss is    0.41.
INFO:tensorflow:For batch 0, accuracy is    0.89.
  64/7352 [..............................] - ETA: 23s - loss: 0.4066 - accuracy: 0.8906INFO:tensorflow:For batch 1, loss is    0.46.
INFO:tensorflow:For batch 1, accuracy is    0.87.
 128/7352 [..............................] - ETA: 22s - loss: 0.4327 - accuracy: 0.8672INFO:tensorflow:For batch 2, loss is    0.52.
INFO:tensorflow:For batch 2, accuracy is    0.84.
 192/7352 [..............................] - ETA: 20s - loss: 0.4605 - accuracy: 0.8385INFO:tensorflow:For batch 3, loss is    0.49.
INFO:tensorflow:For batch 3, accuracy is    0.84.
 256/7352 [>.............................] - ETA: 18s - loss: 0.4667 - accuracy: 0.8398INFO:tensorflow:For batch 4, loss is    0.32.
INFO:tensorflow:For batch 4, accuracy

2816/7352 [==========>...................] - ETA: 13s - loss: 0.3227 - accuracy: 0.8942INFO:tensorflow:For batch 44, loss is    0.21.
INFO:tensorflow:For batch 44, accuracy is    0.90.
2880/7352 [==========>...................] - ETA: 13s - loss: 0.3203 - accuracy: 0.8951INFO:tensorflow:For batch 45, loss is    0.18.
INFO:tensorflow:For batch 45, accuracy is    0.90.
2944/7352 [===========>..................] - ETA: 12s - loss: 0.3173 - accuracy: 0.8967INFO:tensorflow:For batch 46, loss is    0.19.
INFO:tensorflow:For batch 46, accuracy is    0.90.
3008/7352 [===========>..................] - ETA: 12s - loss: 0.3144 - accuracy: 0.8979INFO:tensorflow:For batch 47, loss is    0.34.
INFO:tensorflow:For batch 47, accuracy is    0.90.
3072/7352 [===========>..................] - ETA: 12s - loss: 0.3149 - accuracy: 0.8975INFO:tensorflow:For batch 48, loss is    0.33.
INFO:tensorflow:For batch 48, accuracy is    0.90.
3136/7352 [===========>..................] - ETA: 12s - loss: 0.3153 - accu

5632/7352 [=====================>........] - ETA: 4s - loss: 0.2902 - accuracy: 0.9029INFO:tensorflow:For batch 88, loss is    0.21.
INFO:tensorflow:For batch 88, accuracy is    0.90.
5696/7352 [======================>.......] - ETA: 4s - loss: 0.2893 - accuracy: 0.9033INFO:tensorflow:For batch 89, loss is    0.25.
INFO:tensorflow:For batch 89, accuracy is    0.90.
5760/7352 [======================>.......] - ETA: 4s - loss: 0.2889 - accuracy: 0.9033INFO:tensorflow:For batch 90, loss is    0.21.
INFO:tensorflow:For batch 90, accuracy is    0.90.
5824/7352 [======================>.......] - ETA: 4s - loss: 0.2880 - accuracy: 0.9037INFO:tensorflow:For batch 91, loss is    0.11.
INFO:tensorflow:For batch 91, accuracy is    0.90.
5888/7352 [=======================>......] - ETA: 3s - loss: 0.2861 - accuracy: 0.9044INFO:tensorflow:For batch 92, loss is    0.23.
INFO:tensorflow:For batch 92, accuracy is    0.90.
5952/7352 [=======================>......] - ETA: 3s - loss: 0.2854 - accuracy: 

INFO:tensorflow:weights of 2 layers for epoch4 is []
INFO:tensorflow:weights of 3 layers for epoch4 is [array([[-0.04996726,  0.15325978,  0.02750788, ..., -0.08094353,
         0.01557807,  0.02685143],
       [-0.03489072, -0.15696429, -0.10236818, ...,  0.08197119,
        -0.16556898,  0.07022159],
       [ 0.04911748,  0.2103134 , -0.08593546, ...,  0.14599553,
         0.17170377, -0.02734803],
       ...,
       [ 0.17355657,  0.05241349, -0.22537103, ..., -0.10164401,
         0.03599732,  0.11160706],
       [ 0.05122318,  0.02423824, -0.11206194, ..., -0.12498829,
         0.10656281,  0.00074325],
       [ 0.06873391,  0.16110247, -0.05934371, ...,  0.09850153,
        -0.0411213 ,  0.0534974 ]], dtype=float32), array([ 0.03719037, -0.00435611,  0.117798  ,  0.10981219, -0.02770588,
        0.0717684 ,  0.02734701,  0.03886426,  0.00782489,  0.04921881,
        0.09772774,  0.09246364,  0.0252832 ,  0.01524582,  0.10199089,
        0.00383658,  0.02165579,  0.01761933,  0.13

INFO:tensorflow:The average loss for epoch 4 is    0.27 and accuracy is    0.91.
2947/2947 [==============================] - 4s 1ms/sample - loss: 0.4066 - accuracy: 0.8704
>#1: 87.038
[87.03766465187073]
Accuracy: 87.038% (+/-0.000)
